In [113]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import requests
from bs4 import BeautifulSoup
import lxml
from futsim_funcs import TPR,TFR,TPSR

In [114]:
import warnings
warnings.filterwarnings("ignore", message="The default value of numeric_only in DataFrame.mean is deprecated.", category=FutureWarning)

In [115]:
players_df = pd.read_csv("../fifa24_db/pdb_23.csv")
# match_logs = pd.read_csv("", low_memory=False)
tpr_data = TPR(players_df)
tpr_data.head()

,league_id,club_team_id,league_name,club_name,power,mean_goalkeeping_diving,mean_goalkeeping_handling,mean_goalkeeping_kicking,mean_goalkeeping_positioning,mean_goalkeeping_reflexes,...,mean_power_long_shots,mean_mentality_aggression,mean_mentality_interceptions,mean_mentality_positioning,mean_mentality_vision,mean_mentality_penalties,mean_mentality_composure,mean_defending_marking_awareness,mean_defending_standing_tackle,mean_defending_sliding_tackle
84,13,10,Premier League,Manchester City,85,81.0,78.0,89.0,84.0,86.0,...,69,76,68,71,77,61,83,70,71,63
81,13,9,Premier League,Liverpool,84,86.0,85.0,85.0,90.0,89.0,...,66,78,71,70,75,64,83,68,72,66
30,13,5,Premier League,Chelsea,84,81.0,80.0,83.0,81.0,85.0,...,65,71,67,73,77,62,81,66,69,65
109,53,243,La Liga,Real Madrid,84,84.0,89.0,75.0,89.0,90.0,...,69,71,67,73,76,63,81,66,66,62
70,31,44,Serie A,Inter,83,80.0,80.0,75.0,85.0,84.0,...,66,75,69,67,71,62,81,69,67,64


<br><br><br><br>

# FBRef Match Logs URLs

In [116]:
leagues = ["Premier League", "Serie A", "Ligue 1", "LaLiga", "Bundesliga"]
df = 0

In [137]:
def GetTeamCredentials():
    url = "https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "lxml")

    table = soup.find("table", {"class": "stats_table"})
    team_ids = []
    team_names = []

    for row in table.find_all("tr")[1:]:
        team_name = row.find("td", {"data-stat": "team"}).text
        team_id = row.find("td", {"data-stat": "team"}).a.get("href").split("/")[3]
        team_ids.append(team_id)
        team_names.append(team_name)
    return {
        "team_ids":team_ids,
        "team_names":team_names,
    }

In [156]:
from rapidfuzz import process

def find_best_match(name, choices):
    return process.extractOne(name, choices)

In [157]:
def TeamNameIdConverter(tpr_data, team_name, league_id):
    club_team_names = tpr_data.query(f"league_id == {league_id}")["club_name"].tolist()
    club_team_ids = tpr_data.query(f"league_id == {league_id}")["club_team_id"].tolist()
    club_team_name, score, other = find_best_match(team_name, club_team_names)
    club_team_id = club_team_ids[club_team_names.index(club_team_name)]
    return club_team_id

In [169]:
def FB_URLS(stat_attribute, season="2022-2023"):
    creds = GetTeamCredentials()
    team_ids = creds["team_ids"]
    team_names = creds["team_names"]
    club_team_names = tpr_data.query("league_id == 13")["club_name"].tolist()
    club_team_ids = tpr_data.query("league_id == 13")["club_team_id"].tolist()

    for team_id, team_name in zip(team_ids, team_names):
        best_match = find_best_match(team_name, club_team_names)
        if best_match:
            club_team_name, score, other = best_match
            club_team_id = club_team_ids[club_team_names.index(club_team_name)]
            match_logs_url = f"https://fbref.com/en/squads/{team_id}/{season}/matchlogs/c9/{stat_attribute}/{team_name.replace(' ', '-')}"
            match_logs_stats_dict[stat_attribute].append({
                "id": team_id,
                "stat": stat_attribute,
                "team": team_name,
                "club_team_id": club_team_id,
                "club_team_name": club_team_name, 
                "url": match_logs_url
            })
        else:
            print(f"No match found for team: {team_name}")

In [170]:
match_logs_stats_dict = {
    "shooting":[],
    "passing":[],
    "defense":[],
    "keeper":[],
    "passing_types":[],
    "gca":[],
    "possession":[],
    "misc":[],
}

attributes = match_logs_stats_dict.keys()
for stat_attribute in attributes:
    FB_URLS(stat_attribute)

In [171]:
### <Match Logs' URLS are Ready!>

<br><br><br><br>

# Scraping FBRef

In [181]:
pd.read_html("https://fbref.com/en/squads/b8fd03ef/2022-2023/matchlogs/c9/shooting/-Manchester-City")[0][:38-1]

For Manchester City                                               \
                  Date   Time         Round  Day Venue Result GF GA   
0           2022-08-07  16:30   Matchweek 1  Sun  Away      W  2  0   
1           2022-08-13  15:00   Matchweek 2  Sat  Home      W  4  0   
2           2022-08-21  16:30   Matchweek 3  Sun  Away      D  3  3   
3           2022-08-27  15:00   Matchweek 4  Sat  Home      W  4  2   
4           2022-08-31  19:30   Matchweek 5  Wed  Home      W  6  0   
5           2022-09-03  17:30   Matchweek 6  Sat  Away      D  1  1   
6           2022-09-17  12:30   Matchweek 8  Sat  Away      W  3  0   
7           2022-10-02  14:00   Matchweek 9  Sun  Home      W  6  3   
8           2022-10-08  15:00  Matchweek 10  Sat  Home      W  4  0   
9           2022-10-16  16:30  Matchweek 11  Sun  Away      L  0  1   
10          2022-10-22  15:00  Matchweek 13  Sat  Home      W  3  1   
11          2022-10-29  12:30  Matchweek 14  Sat  Away      W  1  0   
12          2022-11-05  15:00  Matchweek 15  Sat  Home      W  2  1   
13          2022-11-12  12:30  Matchweek 16  Sat  Home      L  1  2   
14          2022-12-28  20:00  Matchweek 17  Wed  Away      W  3  1   
15          2022-12-31  15:00  Matchweek 18  Sat  Home      D  1  1   
16          2023-01-05  20:00  Matchweek 19  Thu  Away      W  1  0   
17          2023-01-14  12:30  Matchweek 20  Sat  Away      L  1  2   
18          2023-01-19  20:00   Matchweek 7  Thu  Home      W  4  2   
19          2023-01-22  14:00  Matchweek 21  Sun  Home      W  3  0   
20          2023-02-05  16:30  Matchweek 22  Sun  Away      L  0  1   
21          2023-02-12  16:30  Matchweek 23  Sun  Home      W  3  1   
22          2023-02-15  19:30  Matchweek 12  Wed  Away      W  3  1   
23          2023-02-18  15:00  Matchweek 24  Sat  Away      D  1  1   
24          2023-02-25  17:30  Matchweek 25  Sat  Away      W  4  1   
25          2023-03-04  12:30  Matchweek 26  Sat  Home      W  2  0   
26          2023-03-11  17:30  Matchweek 27  Sat  Away      W  1  0   
27          2023-04-01  12:30  Matchweek 29  Sat  Home      W  4  1   
28          2023-04-08  17:30  Matchweek 30  Sat  Away      W  4  1   
29          2023-04-15  17:30  Matchweek 31  Sat  Home      W  3  1   
30          2023-04-26  20:00  Matchweek 33  Wed  Home      W  4  1   
31          2023-04-30  14:00  Matchweek 34  Sun  Away      W  2  1   
32          2023-05-03  20:00  Matchweek 28  Wed  Home      W  3  0   
33          2023-05-06  15:00  Matchweek 35  Sat  Home      W  2  1   
34          2023-05-14  14:00  Matchweek 36  Sun  Away      W  3  0   
35          2023-05-21  16:00  Matchweek 37  Sun  Home      W  1  0   
36          2023-05-24  20:00  Matchweek 32  Wed  Away      D  1  1   

                    Standard  ...                   Expected               \
           Opponent      Gls  ...  Dist FK PK PKatt       xG npxG npxG/Sh   
0          West Ham        2  ...  18.7  1  1     1      2.2  1.4    0.11   
1       Bournemouth        3  ...  17.5  0  0     0      1.7  1.7    0.09   
2     Newcastle Utd        3  ...  16.2  1  0     0      2.1  2.1    0.10   
3    Crystal Palace        4  ...  14.1  0  0     0      2.2  2.2    0.13   
4   Nott'ham Forest        6  ...  14.8  0  0     0      3.3  3.3    0.20   
5       Aston Villa        1  ...  16.8  1  0     0      2.1  2.1    0.16   
6            Wolves        3  ...  19.4  0  0     0      1.1  1.1    0.07   
7    Manchester Utd        6  ...  14.2  1  0     0      3.2  3.2    0.15   
8       Southampton        4  ...  13.9  0  0     0      2.4  2.4    0.12   
9         Liverpool        0  ...  18.4  0  0     0      1.0  1.0    0.06   
10         Brighton        3  ...  15.0  0  1     1      1.6  0.8    0.09   
11   Leicester City        1  ...  20.8  1  0     0      1.0  1.0    0.07   
12           Fulham        2  ...  18.1  1  1     1      2.1  1.3    0.09   
13        Brentford        1  ...  20.9  0  0     0      1.6  1.6    0.06   
1